In [1]:
# install dependencies
!pip install pandas sklearn numerapi pycaret

  Using cached scipy-1.5.4-cp38-cp38-manylinux1_x86_64.whl (25.8 MB)
ERROR: yellowbrick 1.3.post1 has requirement numpy<1.20,>=1.16.0, but you'll have numpy 1.20.0 which is incompatible.
ERROR: pycaret 2.3.3 has requirement numpy==1.19.5, but you'll have numpy 1.20.0 which is incompatible.
ERROR: pandas-profiling 3.0.0 has requirement requests>=2.24.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: mljar-supervised 0.10.6 has requirement scikit-learn>=0.24.2, but you'll have scikit-learn 0.23.2 which is incompatible.
ERROR: mljar-supervised 0.10.6 has requirement scipy>=1.6.1, but you'll have scipy 1.5.4 which is incompatible.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.1
    Uninstalling scipy-1.7.1:
      Successfully uninstalled scipy-1.7.1


In [2]:
# import dependencies
import pandas as pd
import numerapi
import sklearn.linear_model

In [3]:
# download the latest tournament dataset (takes around 30s)
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")

In [4]:
# find only the feature columns
feature_cols = tournament_data.columns[tournament_data.columns.str.startswith('feature')]
feature_cols = feature_cols.tolist()

In [5]:
# predict the target on the live features
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor.load("AutogluonModels/ag-20210905_162148/")

In [6]:
df_reader = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz", chunksize=4096)

In [ ]:
y_pred = []
for df_chunk in df_reader:
    y_pred.append(predictor.predict(df_chunk[feature_cols], as_pandas=True))
y_pred = pd.concat(y_pred, axis=0, ignore_index=True)

In [ ]:
# predictions must have an `id` column and a `prediction` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction"] = y_pred
predictions_df.head()